In [1]:
from bs4 import BeautifulSoup as bs
import lxml
import requests
import re
import os
from datetime import datetime

In [3]:
pattern = 'value="\d"'

with os.scandir("./html/") as entries:
    
    with open('tg.csv', 'w') as csv_out, open('tg.txt', 'w') as hodoku_out, open('gsheet.csv', 'w') as gout:
    
        for file in entries:
            input_file = file
            file_name = file.name


            with open(input_file) as html_file:

                contenido = html_file.read()
                soup = bs(contenido, 'lxml')

                # Fecha
                
                try:

                    fecha = soup.find('div', class_='f-f-and-m').text.split(' ')
                    weekday = fecha[0]
                    ordDay = fecha[1]
                    month = fecha[2]
                    year = fecha[3]
                    day = str(ordDay).replace('th', '')
                    day = day.replace('st', '')
                    day = day.replace('rd', '')
                    day = day.replace('nd', '')
                    date = f'{weekday}, {day} {month} {year}'
                    date2 = datetime.strptime(date, '%A, %d %B %Y')
                    date3 = f'{date2.year}-{str(date2.month).zfill(2)}-{str(date2.day).zfill(2)}'
                    
                except:
                    
                    print(file_name)
                    continue

                # Dificultad

                dificultad = soup.find('div', class_='game-type').text.replace('Sudoku - ', '').split(' ')
                dificultad = dificultad[1]
                
                # Dificultad nombre
                
                if dificultad == 'Diabolical':
                    dif = 'DIAB'
                if dificultad == 'Tough':
                    dif = 'TGH'

                # Numero Sudoku

                listaNumSudoku = soup.find('span', class_='game-id').text.split(' ')
                numeroSudoku = listaNumSudoku[1].replace(',', '')

                # Tabla Puzzle

                # Las coordenadas representan Columna,Fila

                cadena = []

                tablaPuzzle = soup.find('table', attrs={'id': 'puzzle'})
                for row in tablaPuzzle:
                    tableRow = tablaPuzzle.find_all('tr')
                    for row in tableRow:
                        for cell in row:
                            value = re.search(pattern, str(cell))
                            if value == None:
                                value = 0
                            else:
                                value = value.group(0).replace('value=', '')
                                value = value.replace('"', '')
                                value = int(value)
                            cadena.append(str(value))

                    sudoku = ''.join(cadena)
                    
                    csv_out.write(f'{sudoku}|{weekday}, {ordDay} {month} {year}|{numeroSudoku}|{dificultad}\n')
                    hodoku_out.write(f'{sudoku}\n')
                    gout.write(f'{sudoku}|{str(date2.day).zfill(2)}|{str(date2.month).zfill(2)}|{date2.year}|{numeroSudoku}||||TG|{dificultad}|Telegraph\n')